In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from faces import detect_faces
import numpy as np
import argparse
import imutils
import time
import cv2
import pickle
import os

print("Imports Successful!!")


Imports Successful!!


In [27]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/face_detector/"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_recognizer_exp_r1_n8_g8.yml"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_label_encoder_exp_r1_n8_g8.pickle"


In [28]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([detector_main_folder, "deploy.prototxt"])
weightsPath = os.path.sep.join([detector_main_folder, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

recognizer_path = recognizer_save_path
label_encoder_path = label_encoder_save_path
print("[INFO] loading trained face recognizer model...")
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(recognizer_path)
print("[INFO] loaded trained face recognizer model...")

le = pickle.loads(open(label_encoder_path, "rb").read())
print("[INFO] loaded trained label encoder...")

# Get input feed
feed = 0
cam = cv2.VideoCapture(feed)
fps = []
confs = []
num_frames = 1
correct = 0
while True:
    ret, frame = cam.read()
    time_1 = time.time()
    boxes = detect_faces(net, frame)
#     print(boxes, time.time())
    image = frame.copy()
    # loop over the bounding boxes
    faces = []
    for (startX, startY, endX, endY) in boxes:
        # extract the face ROI, resize it, and convert it to
        # grayscale
        faceROI = image[startY:endY, startX:endX].copy()
        faceROI = cv2.resize(faceROI, (47, 62))
        faceROI = cv2.cvtColor(faceROI, cv2.COLOR_BGR2GRAY)
        faces.append(faceROI)
        
    for face in faces:
        (prediction, conf) = recognizer.predict(face)
        predName = le.inverse_transform([prediction])[0]
        if predName == "shubhranshu_malhotra":
            correct+=1
        text = "{}: {:.2f}%".format(predName, conf)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY),
                        (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,0), 2)
        confs.append(conf)
    

    cv2.imshow("detector", image)
    time_2 = time.time()
    fps.append(1/(time_2 - time_1))
    if num_frames == 100:        
        print(f"Accuracy: {correct}")
        print(f"Average confidence: {np.mean(confs):.2f}")
        print(f"Average FPS: {np.mean(fps):.2f}")
        correct = 0
        confs = []
        fps = []
        num_frames = 0
    num_frames += 1
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()



[INFO] loading face detector model...
[INFO] loading trained face recognizer model...
[INFO] loaded trained face recognizer model...
[INFO] loaded trained label encoder...
Accuracy: 100
Average confidence: 145.15
Average FPS: 15.01
Accuracy: 100
Average confidence: 151.44
Average FPS: 15.11
Accuracy: 100
Average confidence: 143.20
Average FPS: 15.42
Accuracy: 99
Average confidence: 147.86
Average FPS: 15.00
Accuracy: 84
Average confidence: 152.07
Average FPS: 16.35
Accuracy: 99
Average confidence: 138.83
Average FPS: 15.13
Accuracy: 94
Average confidence: 144.41
Average FPS: 15.50
Accuracy: 98
Average confidence: 150.14
Average FPS: 15.29
Accuracy: 98
Average confidence: 148.54
Average FPS: 15.67
Accuracy: 89
Average confidence: 154.41
Average FPS: 14.98
Accuracy: 97
Average confidence: 142.24
Average FPS: 15.33
Accuracy: 82
Average confidence: 144.69
Average FPS: 15.78
Accuracy: 72
Average confidence: 151.58
Average FPS: 15.94
Accuracy: 90
Average confidence: 153.60
Average FPS: 15.51

**Note:** To get an idea of how many frames face was missed check the difference in frame rate of `Near` vs `Mid_Far`. More the missed faces higher the FPS.  


### recognizer  [radius=1, neighbors=8, grid_x=8, grid_y=8] <<BEST\>>

`Near:`  
* Accuracy: 99
* Average confidence: 147.86
* Average FPS: 15.00

`Mid:`  
* Accuracy: 98
* Average confidence: 150.14
* Average FPS: 15.29

`Mid_Far:`  
* Accuracy: 86
* Average confidence: 158.20
* Average FPS: 15.15



## Experiments


### [radius=1, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 100
* Average confidence: 62.98
* Average FPS: 16.39

`Mid:`  
* Accuracy: 95
* Average confidence: 66.04
* Average FPS: 16.72

`Mid_Far:`  
* Accuracy: 80
* Average confidence: 70.83
* Average FPS: 17.12


### [radius=1, neighbors=8, grid_x=8, grid_y=8]

`Near:`  
* Accuracy: 99
* Average confidence: 147.86
* Average FPS: 15.00

`Mid:`  
* Accuracy: 98
* Average confidence: 150.14
* Average FPS: 15.29

`Mid_Far:`  
* Accuracy: 86
* Average confidence: 158.20
* Average FPS: 15.15


### [radius=2, neighbors=8, grid_x=6, grid_y=6]

`Near:`  

* Accuracy: 99
* Average confidence: 74.76  
* Average FPS: 16.95  

`Mid:`  
* Accuracy: 83
* Average FPS: 16
* Average confidence: 80

`Mid_Far:`  
* Accuracy: 35
* Average FPS: 16.43
* Average confidence: 76.92


### [radius=3, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 85
* Average confidence: 87.27
* Average FPS: 16.42

`Mid:`  
* Accuracy: 77
* Average confidence: 85.96
* Average FPS: 16.33

`Mid_Far:`  
* Accuracy: 62
* Average confidence: 87.31
* Average FPS: 16.49



### [radius=10, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 70
* Average confidence: 97.14
* Average FPS: 18.37

`Mid:`  
* Accuracy: 56
* Average confidence: 97.92
* Average FPS: 17.53

`Mid_Far:`  
* Accuracy: 25
* Average confidence: 101.57
* Average FPS: 18.10



### [radius=1, neighbors=12, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 99
* Average confidence: 82.36
* Average FPS: 8.08

`Mid:`  
* Accuracy: 100
* Average confidence: 83.31
* Average FPS: 8.32

`Mid_Far:`  
* Accuracy: 73
* Average confidence: 84.36
* Average FPS: 8.28


### [radius=1, neighbors=12, grid_x=7, grid_y=7]

`Near:`  
* Accuracy: 100
* Average confidence: 125.96
* Average FPS: 6.75

`Mid:`  
* Accuracy: 93
* Average confidence: 125.66
* Average FPS: 7.96

`Mid_Far:`  
* Accuracy: 56
* Average confidence: 128.63
* Average FPS: 8.30


### [radius=1, neighbors=12, grid_x=8, grid_y=8]

`Near:`  
* Accuracy: 99
* Average confidence: 174.30
* Average FPS: 5.93

`Mid:`  
* Accuracy: 95
* Average confidence: 175.50
* Average FPS: 5.81

`Mid_Far:`  
* Accuracy: 60
* Average confidence: 182.13
* Average FPS: 9.73


### [radius=2, neighbors=12, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 99
* Average confidence: 93.37
* Average FPS: 7.98

`Mid:`  
* Accuracy: 87
* Average confidence: 101.56
* Average FPS: 9.24

`Mid_Far:`  
* Accuracy: 48
* Average confidence: 100.79
* Average FPS: 10.07


### [radius=2, neighbors=12, grid_x=7, grid_y=7]

`Near:`  
* Accuracy: 98
* Average confidence: 140.65
* Average FPS: 6.61

`Mid:`  
* Accuracy: 85
* Average confidence: 146.17
* Average FPS: 7.92

`Mid_Far:`  
* Accuracy: 45
* Average confidence: 147.27
* Average FPS: 12.89

### [radius=2, neighbors=12, grid_x=8, grid_y=8]

`Near:`  
* Accuracy: 99
* Average confidence: 195.09
* Average FPS: 5.67

`Mid:`  
* Accuracy: 90
* Average confidence: 195.92
* Average FPS: 5.78

`Mid_Far:`  
* Accuracy: 50
* Average confidence: 197.23
* Average FPS: 7.33


### [radius=1, neighbors=16, grid_x=6, grid_y=6]

***`CRASHED`***
